# Alpha Pods Take-Home Assignment Analysis and Execution Plan

I'll analyze each task at a high level before diving into detailed implementation for Task A.

## High-Level Analysis of All Tasks

### Task A - Baseline CVaR Index (Required)
**Objective**: Rebuild a long-only index optimized for 95% daily Conditional Value-at-Risk (CVaR)
**Key Components**:
- Portfolio optimization with CVaR objective
- Quarterly rebalancing with constraints
- Transaction cost modeling
- Benchmarking against equal-weighted and cap-weighted indices

**Approach**: 
1. Data collection for 60 liquid S&P 100 stocks
2. CVaR optimization implementation
3. Backtesting framework
4. Performance metrics calculation

### Task B - Alpha Enhancement (Required)
**Objective**: Improve risk-adjusted returns using ML/AI/statistical methods
**Potential Approaches**:
- Tail-risk modeling with quantile regression
- Macro regime adaptation
- Factor-based overlays
- Dynamic rebalancing with RL

**Key Considerations**:
- Walk-forward validation
- Interpretability requirements
- Public data only constraint

### Task C - Alpha in the Wild (Optional)
**Objective**: Incorporate non-traditional data sources
**Potential Data Sources**:
- News sentiment analysis
- Google Trends data
- Social media metrics
- Microstructure signals

## Detailed Plan for Task A

### 1. Data Preparation

**Data Requirements**:
- Daily price data for 60 S&P 100 stocks (Jan 2010 - Dec 2024)
- Market cap data for benchmark construction
- Risk-free rate for Sharpe ratio calculation

**Data Sources**:
- Yahoo Finance (yfinance)
- Quandl or Bloomberg if available
- FRED for risk-free rates

### 2. CVaR Optimization Implementation

**Mathematical Formulation**:
```
Minimize: CVaR_α(w) 
Subject to:
  Σ w_i = 1 (fully invested)
  0 ≤ w_i ≤ 0.05 (no shorting, max 5% per stock)
```

**Implementation Steps**:
1. Calculate daily returns matrix
2. Implement CVaR calculation using historical simulation
3. Set up convex optimization problem (using cvxpy)
4. Solve with quarterly rebalancing

### 3. Backtesting Framework

**Components**:
- Rebalancing schedule (quarterly)
- Transaction cost model (10bps per trade)
- Portfolio return calculation
- Index value computation

### 4. Performance Metrics

**Required Metrics**:
- Annualized return and volatility
- Sharpe ratio
- 95% CVaR
- Maximum drawdown
- Turnover

### 5. Benchmarking

**Benchmarks to Include**:
- Equal-weighted portfolio
- Cap-weighted portfolio
- Possibly SPY as market benchmark

## Python Implementation Plan

Let's start prototyping the Python implementation for Task A.

```python
# Import required libraries
import numpy as np
import pandas as pd
import yfinance as yf
import cvxpy as cp
from datetime import datetime
import matplotlib.pyplot as plt

# 1. Data Collection Function
def fetch_stock_data(tickers, start_date, end_date):
    """
    Fetch historical stock data from Yahoo Finance
    Args:
        tickers (list): List of stock tickers
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format
    Returns:
        pd.DataFrame: DataFrame with adjusted close prices
    """
    print(f"Downloading data for {len(tickers)} stocks from {start_date} to {end_date}")
    data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
    return data

# 2. CVaR Calculation
def calculate_cvar(returns, alpha=0.95):
    """
    Calculate Conditional Value-at-Risk (CVaR)
    Args:
        returns (np.array): Array of portfolio returns
        alpha (float): Confidence level (0.95 for 95% CVaR)
    Returns:
        float: CVaR value
    """
    var = np.percentile(returns, 100*(1-alpha))
    cvar = returns[returns <= var].mean()
    return cvar

# 3. Portfolio Optimization
def optimize_cvar_portfolio(returns, alpha=0.95, max_weight=0.05):
    """
    Optimize portfolio weights to minimize CVaR
    Args:
        returns (pd.DataFrame): DataFrame of historical returns
        alpha (float): Confidence level for CVaR
        max_weight (float): Maximum weight per stock
    Returns:
        np.array: Optimal weights
    """
    n = returns.shape[1]
    weights = cp.Variable(n)
    
    # Historical scenarios
    scenario_returns = returns.values
    portfolio_returns = scenario_returns @ weights
    
    # CVaR calculation using Rockafellar-Uryasev formulation
    beta = cp.Variable()
    tau = cp.Variable()
    loss = -portfolio_returns
    cvar = tau + (1/(1-alpha)) * cp.mean(cp.pos(loss - tau))
    
    # Problem constraints
    constraints = [
        cp.sum(weights) == 1,
        weights >= 0,
        weights <= max_weight
    ]
    
    # Solve optimization problem
    problem = cp.Problem(cp.Minimize(cvar), constraints)
    problem.solve()
    
    return weights.value

# 4. Backtesting Engine
def run_backtest(price_data, rebalance_freq='Q', transaction_cost=0.001):
    """
    Run backtest of CVaR-optimized strategy
    Args:
        price_data (pd.DataFrame): DataFrame of historical prices
        rebalance_freq (str): Rebalancing frequency ('Q' for quarterly)
        transaction_cost (float): Transaction cost per trade (10bps = 0.001)
    Returns:
        dict: Backtest results including portfolio values and metrics
    """
    returns = price_data.pct_change().dropna()
    dates = returns.index
    rebalance_dates = pd.date_range(dates[0], dates[-1], freq=rebalance_freq)
    
    # Initialize portfolio
    n_stocks = returns.shape[1]
    current_weights = np.ones(n_stocks) / n_stocks  # Start with equal weights
    portfolio_value = [1.0]  # Starting with $1
    weights_history = []
    
    for i in range(1, len(dates)):
        date = dates[i]
        
        # Check if rebalance needed
        if date in rebalance_dates:
            # Use lookback window of 1 year for optimization
            lookback_returns = returns.loc[:date].iloc[-252:]
            
            # Get optimal weights
            new_weights = optimize_cvar_portfolio(lookback_returns)
            
            # Apply transaction costs
            turnover = np.sum(np.abs(new_weights - current_weights))
            portfolio_value[-1] *= (1 - transaction_cost * turnover)
            
            current_weights = new_weights
        
        # Record daily portfolio return
        daily_return = np.dot(returns.iloc[i], current_weights)
        portfolio_value.append(portfolio_value[-1] * (1 + daily_return))
        weights_history.append(current_weights)
    
    results = {
        'dates': dates,
        'portfolio_value': portfolio_value,
        'weights': weights_history
    }
    return results

# 5. Performance Metrics Calculation
def calculate_performance_metrics(portfolio_values, benchmark_values, risk_free_rate=0.0):
    """
    Calculate performance metrics
    Args:
        portfolio_values (list): List of portfolio values over time
        benchmark_values (dict): Dictionary of benchmark values
        risk_free_rate (float): Annual risk-free rate
    Returns:
        dict: Dictionary of performance metrics
    """
    # Convert to returns
    port_returns = pd.Series(portfolio_values).pct_change().dropna()
    
    # Annualized return
    annual_return = (1 + port_returns.mean())**252 - 1
    
    # Annualized volatility
    annual_vol = port_returns.std() * np.sqrt(252)
    
    # Sharpe ratio
    sharpe = (annual_return - risk_free_rate) / annual_vol
    
    # Maximum drawdown
    cumulative = (1 + port_returns).cumprod()
    peak = cumulative.expanding(min_periods=1).max()
    drawdown = (cumulative - peak) / peak
    max_drawdown = drawdown.min()
    
    # CVaR
    cvar_95 = calculate_cvar(port_returns)
    
    metrics = {
        'Annual Return': annual_return,
        'Annual Volatility': annual_vol,
        'Sharpe Ratio': sharpe,
        '95% CVaR': cvar_95,
        'Max Drawdown': max_drawdown
    }
    
    return metrics

# Main execution
if __name__ == "__main__":
    # Configuration
    start_date = '2010-01-01'
    end_date = '2024-12-31'
    
    # Sample 60 liquid S&P 100 stocks (replace with actual list)
    sp100_tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'FB', 'JNJ', 'JPM', 'V', 'PG', 'HD']
    
    # 1. Fetch data
    price_data = fetch_stock_data(sp100_tickers, start_date, end_date)
    
    # 2. Run backtest
    results = run_backtest(price_data)
    
    # 3. Calculate performance metrics
    metrics = calculate_performance_metrics(results['portfolio_value'], {})
    
    # 4. Create benchmark (equal-weighted)
    ew_returns = price_data.pct_change().mean(axis=1).dropna()
    ew_values = (1 + ew_returns).cumprod()
    
    # 5. Plot results
    plt.figure(figsize=(12, 6))
    plt.plot(results['dates'], results['portfolio_value'], label='CVaR Optimized')
    plt.plot(ew_returns.index, ew_values, label='Equal Weighted')
    plt.title('Portfolio Performance Comparison')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value')
    plt.legend()
    plt.grid()
    plt.show()
    
    # 6. Print metrics
    print("Performance Metrics:")
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}")
```

## Next Steps for Task A Completion

1. **Finalize the stock universe**: Identify the 60 liquid S&P 100 stocks to include
2. **Enhance the data pipeline**: Add proper error handling for data fetching
3. **Improve optimization stability**: Add regularization if needed
4. **Complete benchmarking**: Add cap-weighted benchmark
5. **Output generation**: Create CSV with daily index values and performance table
6. **Validation**: Cross-check results against expected behavior from the paper

Would you like me to elaborate on any particular aspect of this Task A implementation before we proceed to Task B?

In [0]:
!pip install cvxpy

In [0]:
# Import required libraries
import numpy as np
import pandas as pd
import yfinance as yf
import cvxpy as cp
from datetime import datetime
import matplotlib.pyplot as plt

# 1. Data Collection Function
def fetch_stock_data(tickers, start_date, end_date):
    """
    Fetch historical stock data from Yahoo Finance
    Args:
        tickers (list): List of stock tickers
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format
    Returns:
        pd.DataFrame: DataFrame with adjusted close prices
    """
    print(f"Downloading data for {len(tickers)} stocks from {start_date} to {end_date}")
    data = yf.download(tickers, start=start_date, end=end_date)['Close']
    return data

# 2. CVaR Calculation
def calculate_cvar(returns, alpha=0.95):
    """
    Calculate Conditional Value-at-Risk (CVaR)
    Args:
        returns (np.array): Array of portfolio returns
        alpha (float): Confidence level (0.95 for 95% CVaR)
    Returns:
        float: CVaR value
    """
    var = np.percentile(returns, 100*(1-alpha))
    cvar = returns[returns <= var].mean()
    return cvar


# 3. Portfolio Optimization
def optimize_cvar_portfolio(returns, alpha=0.95, max_weight=0.05):
    """
    Optimize portfolio weights to minimize CVaR
    Args:
        returns (pd.DataFrame): DataFrame of historical returns
        alpha (float): Confidence level for CVaR
        max_weight (float): Maximum weight per stock
    Returns:
        np.array: Optimal weights (falls back to equal weights if optimization fails)
    """
    # Validate input
    if returns.isnull().values.any():
        print("Warning: Input returns contain NaN values - filling with zeros")
        returns = returns.fillna(0)
    
    if len(returns) < 10:  # Insufficient data points
        print("Warning: Insufficient data points - returning equal weights")
        n = returns.shape[1]
        return np.ones(n) / n
    
    n = returns.shape[1]
    weights = cp.Variable(n)
    
    # Historical scenarios
    scenario_returns = returns.values
    portfolio_returns = scenario_returns @ weights
    
    # CVaR calculation using Rockafellar-Uryasev formulation
    beta = cp.Variable()
    tau = cp.Variable()
    loss = -portfolio_returns
    cvar = tau + (1/(1-alpha)) * cp.mean(cp.pos(loss - tau))
    
    # Problem constraints
    constraints = [
        cp.sum(weights) == 1,
        weights >= 0,
        weights <= max_weight
    ]
    
    # Solve optimization problem
    problem = cp.Problem(cp.Minimize(cvar), constraints)
    
    try:
        problem.solve(solver=cp.ECOS, verbose=False)
        if weights.value is None:
            raise Exception("Solver failed to find solution")
        
        # Normalize weights in case of small numerical errors
        optimized_weights = np.maximum(weights.value, 0)
        optimized_weights = optimized_weights / optimized_weights.sum()
        return optimized_weights
        
    except Exception as e:
        print(f"Optimization failed: {str(e)} - using equal weights as fallback")
        return np.ones(n) / n

# 4. Backtesting Engine
def run_backtest(price_data, rebalance_freq='Q', transaction_cost=0.001):
    """
    Run backtest of CVaR-optimized strategy
    Args:
        price_data (pd.DataFrame): DataFrame of historical prices
        rebalance_freq (str): Rebalancing frequency ('Q' for quarterly)
        transaction_cost (float): Transaction cost per trade (10bps = 0.001)
    Returns:
        dict: Backtest results including portfolio values and metrics
    """
    # Calculate returns and clean data
    returns = price_data.pct_change().dropna()
    returns = returns.replace([np.inf, -np.inf], np.nan).fillna(0)
    
    dates = returns.index
    rebalance_dates = pd.date_range(dates[0], dates[-1], freq=rebalance_freq)
    
    # Initialize portfolio
    n_stocks = returns.shape[1]
    current_weights = np.ones(n_stocks) / n_stocks  # Start with equal weights
    portfolio_value = [1.0]  # Starting with $1
    weights_history = []
    
    for i in range(1, len(dates)):
        date = dates[i]
        
        # Check if rebalance needed
        if date in rebalance_dates:
            # Use lookback window of 1 year for optimization
            lookback_returns = returns.loc[:date].iloc[-252:] if len(returns.loc[:date]) >= 252 else returns.loc[:date]
            
            # Get optimal weights
            new_weights = optimize_cvar_portfolio(lookback_returns)
            
            # Apply transaction costs
            turnover = np.sum(np.abs(new_weights - current_weights))
            portfolio_value[-1] *= (1 - transaction_cost * turnover)
            
            current_weights = new_weights
        
        # Record daily portfolio return
        daily_return = np.dot(returns.iloc[i], current_weights)
        portfolio_value.append(portfolio_value[-1] * (1 + daily_return))
        weights_history.append(current_weights)
    
    results = {
        'dates': dates,
        'portfolio_value': portfolio_value,
        'weights': weights_history
    }
    return results


# 5. Performance Metrics Calculation
def calculate_performance_metrics(portfolio_values, benchmark_values, risk_free_rate=0.0):
    """
    Calculate performance metrics
    Args:
        portfolio_values (list): List of portfolio values over time
        benchmark_values (dict): Dictionary of benchmark values
        risk_free_rate (float): Annual risk-free rate
    Returns:
        dict: Dictionary of performance metrics
    """
    # Convert to returns
    port_returns = pd.Series(portfolio_values).pct_change().dropna()
    
    # Annualized return
    annual_return = (1 + port_returns.mean())**252 - 1
    
    # Annualized volatility
    annual_vol = port_returns.std() * np.sqrt(252)
    
    # Sharpe ratio
    sharpe = (annual_return - risk_free_rate) / annual_vol
    
    # Maximum drawdown
    cumulative = (1 + port_returns).cumprod()
    peak = cumulative.expanding(min_periods=1).max()
    drawdown = (cumulative - peak) / peak
    max_drawdown = drawdown.min()
    
    # CVaR
    cvar_95 = calculate_cvar(port_returns)
    
    metrics = {
        'Annual Return': annual_return,
        'Annual Volatility': annual_vol,
        'Sharpe Ratio': sharpe,
        '95% CVaR': cvar_95,
        'Max Drawdown': max_drawdown
    }
    
    return metrics



In [0]:
# Main execution
if __name__ == "__main__":
    # Configuration
    start_date = '2010-01-01'
    end_date = '2024-12-31'
    
    # Sample 60 liquid S&P 100 stocks (replace with actual list)
    sp100_tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'JNJ', 'JPM', 'V', 'PG']
    
    # 1. Fetch data
    price_data = fetch_stock_data(sp100_tickers, start_date, end_date)
    
    # 2. Run backtest
    results = run_backtest(price_data)
    
    # 3. Calculate performance metrics
    metrics = calculate_performance_metrics(results['portfolio_value'], {})
    
    # 4. Create benchmark (equal-weighted)
    ew_returns = price_data.pct_change().mean(axis=1).dropna()
    ew_values = (1 + ew_returns).cumprod()
    
    # 5. Plot results
    plt.figure(figsize=(12, 6))
    plt.plot(results['dates'], results['portfolio_value'], label='CVaR Optimized')
    plt.plot(ew_returns.index, ew_values, label='Equal Weighted')
    plt.title('Portfolio Performance Comparison')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value')
    plt.legend()
    plt.grid()
    plt.show()
    
    # 6. Print metrics
    print("Performance Metrics:")
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}")

In [0]:
# Import required libraries
import numpy as np
import pandas as pd
import yfinance as yf
import cvxpy as cp
from datetime import datetime
import matplotlib.pyplot as plt
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# 1. Data Collection and Preparation
def fetch_stock_data(tickers, start_date, end_date):
    """
    Fetch historical stock data from Yahoo Finance with enhanced error handling
    Args:
        tickers (list): List of stock tickers
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format
    Returns:
        pd.DataFrame: Cleaned DataFrame with adjusted close prices
    """
    print(f"Downloading data for {len(tickers)} stocks from {start_date} to {end_date}")
    
    try:
        # Download data with progress bar
        data = yf.download(tickers, start=start_date, end=end_date, progress=True)['Close']
        
        # Check if any stocks failed to download
        if data.isnull().all().any():
            failed_tickers = data.columns[data.isnull().all()].tolist()
            print(f"Warning: Failed to download data for {failed_tickers} - removing from universe")
            data = data.drop(columns=failed_tickers)
        
        # Forward fill missing values and drop any remaining NAs
        data = data.ffill().dropna()
        
        # Verify we have sufficient data
        if len(data) < 252:  # Less than 1 year of data
            raise ValueError("Insufficient data points after cleaning")
            
        print("Data download and cleaning completed successfully")
        return data
    
    except Exception as e:
        print(f"Error in data download: {str(e)}")
        raise

# 2. Enhanced CVaR Optimization
def optimize_cvar_portfolio(returns, alpha=0.95, max_weight=0.05, verbose=False):
    """
    Optimize portfolio weights to minimize CVaR with robust error handling
    Args:
        returns (pd.DataFrame): DataFrame of historical returns
        alpha (float): Confidence level for CVaR
        max_weight (float): Maximum weight per stock
        verbose (bool): Whether to print optimization details
    Returns:
        np.array: Optimal weights (falls back to equal weights if optimization fails)
    """
    # Validate input
    if returns.isnull().values.any():
        if verbose:
            print("Warning: Input returns contain NaN values - filling with zeros")
        returns = returns.fillna(0)
    
    if len(returns) < 10:  # Insufficient data points
        if verbose:
            print("Warning: Insufficient data points - returning equal weights")
        n = returns.shape[1]
        return np.ones(n) / n
    
    n = returns.shape[1]
    weights = cp.Variable(n)
    
    # Historical scenarios
    scenario_returns = returns.values
    portfolio_returns = scenario_returns @ weights
    
    # CVaR calculation using Rockafellar-Uryasev formulation
    beta = cp.Variable()
    tau = cp.Variable()
    loss = -portfolio_returns
    cvar = tau + (1/(1-alpha)) * cp.mean(cp.pos(loss - tau))
    
    # Problem constraints
    constraints = [
        cp.sum(weights) == 1,
        weights >= 0,
        weights <= max_weight
    ]
    
    # Solve optimization problem
    problem = cp.Problem(cp.Minimize(cvar), constraints)
    
    try:
        # Try multiple solvers for robustness
        solvers = [cp.ECOS, cp.SCS, cp.CVXOPT]
        solution_found = False
        
        for solver in solvers:
            try:
                problem.solve(solver=solver, verbose=verbose)
                if weights.value is not None:
                    solution_found = True
                    break
            except:
                continue
                
        if not solution_found:
            raise Exception("All solvers failed to find solution")
        
        # Process optimized weights
        optimized_weights = np.maximum(weights.value, 0)  # Ensure non-negative
        optimized_weights = optimized_weights / optimized_weights.sum()  # Re-normalize
        
        if verbose:
            print("Optimization successful")
            print(f"Optimal weights: {optimized_weights}")
            print(f"Sum of weights: {optimized_weights.sum():.6f}")
        
        return optimized_weights
        
    except Exception as e:
        if verbose:
            print(f"Optimization failed: {str(e)} - using equal weights as fallback")
        return np.ones(n) / n

# 3. Robust Backtesting Engine
def run_backtest(price_data, rebalance_freq='Q', transaction_cost=0.001, verbose=False):
    """
    Enhanced backtest of CVaR-optimized strategy with detailed logging
    Args:
        price_data (pd.DataFrame): DataFrame of historical prices
        rebalance_freq (str): Rebalancing frequency ('Q' for quarterly)
        transaction_cost (float): Transaction cost per trade (10bps = 0.001)
        verbose (bool): Whether to print detailed backtest information
    Returns:
        dict: Backtest results including portfolio values and metrics
    """
    # Calculate returns and clean data
    returns = price_data.pct_change().dropna()
    returns = returns.replace([np.inf, -np.inf], np.nan).fillna(0)
    
    if verbose:
        print("\nReturns statistics:")
        print(returns.describe())
    
    dates = returns.index
    rebalance_dates = pd.date_range(dates[0], dates[-1], freq=rebalance_freq)
    
    # Initialize portfolio
    n_stocks = returns.shape[1]
    current_weights = np.ones(n_stocks) / n_stocks  # Start with equal weights
    portfolio_value = [1.0]  # Starting with $1
    weights_history = []
    rebalance_log = []
    
    for i in range(1, len(dates)):
        date = dates[i]
        
        # Check if rebalance needed
        if date in rebalance_dates:
            # Use lookback window of 1 year for optimization
            lookback_window = min(252, len(returns.loc[:date]))
            lookback_returns = returns.loc[:date].iloc[-lookback_window:]
            
            if verbose:
                print(f"\nRebalancing on {date.strftime('%Y-%m-%d')}")
                print(f"Using {len(lookback_returns)} days of returns data")
            
            # Get optimal weights
            new_weights = optimize_cvar_portfolio(lookback_returns, verbose=verbose)
            
            # Calculate and apply transaction costs
            turnover = np.sum(np.abs(new_weights - current_weights))
            cost = transaction_cost * turnover
            portfolio_value[-1] *= (1 - cost)
            
            if verbose:
                print(f"Turnover: {turnover:.4f}, Transaction cost: {cost:.4%}")
                print(f"Portfolio value before rebalance: {portfolio_value[-1]:.4f}")
            
            current_weights = new_weights
            rebalance_log.append({
                'date': date,
                'weights': current_weights,
                'turnover': turnover,
                'cost': cost
            })
        
        # Record daily portfolio return
        daily_return = np.dot(returns.iloc[i], current_weights)
        portfolio_value.append(portfolio_value[-1] * (1 + daily_return))
        weights_history.append(current_weights)
    
    results = {
        'dates': dates,
        'portfolio_value': portfolio_value,
        'weights': weights_history,
        'rebalance_log': pd.DataFrame(rebalance_log),
        'returns': returns
    }
    return results

# 4. Comprehensive Performance Metrics
def calculate_performance_metrics(portfolio_values, benchmark_values=None, risk_free_rate=0.0):
    """
    Calculate detailed performance metrics with benchmark comparison
    Args:
        portfolio_values (list): List of portfolio values over time
        benchmark_values (dict): Dictionary of benchmark values
        risk_free_rate (float): Annual risk-free rate
    Returns:
        dict: Dictionary of performance metrics
    """
    # Convert to returns
    port_returns = pd.Series(portfolio_values).pct_change().dropna()
    
    # Calculate metrics
    metrics = {}
    
    # Basic metrics
    metrics['Cumulative Return'] = portfolio_values[-1] / portfolio_values[0] - 1
    metrics['Annualized Return'] = (1 + port_returns.mean())**252 - 1
    metrics['Annualized Volatility'] = port_returns.std() * np.sqrt(252)
    metrics['Sharpe Ratio'] = (metrics['Annualized Return'] - risk_free_rate) / metrics['Annualized Volatility']
    
    # Risk metrics
    metrics['95% VaR'] = np.percentile(port_returns, 5)
    metrics['95% CVaR'] = port_returns[port_returns <= metrics['95% VaR']].mean()
    
    # Drawdown analysis
    cumulative = (1 + port_returns).cumprod()
    peak = cumulative.expanding(min_periods=1).max()
    drawdown = (cumulative - peak) / peak
    metrics['Max Drawdown'] = drawdown.min()
    metrics['Avg Drawdown'] = drawdown.mean()
    metrics['Drawdown Duration (days)'] = (drawdown < 0).astype(int).groupby((drawdown < 0).astype(int).diff().ne(0).cumsum()).max()
    
    # Benchmark comparison if provided
    if benchmark_values:
        for bench_name, bench_values in benchmark_values.items():
            bench_returns = pd.Series(bench_values).pct_change().dropna()
            metrics[f'Excess Return vs {bench_name}'] = metrics['Annualized Return'] - ((1 + bench_returns.mean())**252 - 1)
            metrics[f'Tracking Error vs {bench_name}'] = (port_returns - bench_returns).std() * np.sqrt(252)
    
    return metrics

# 5. Visualization and Reporting
def generate_report(results, price_data):
    """
    Generate comprehensive performance report with visualizations
    Args:
        results (dict): Backtest results from run_backtest
        price_data (pd.DataFrame): Original price data
    """
    # Create figure
    plt.figure(figsize=(15, 10))
    
    # Portfolio performance plot
    plt.subplot(2, 2, 1)
    plt.plot(results['dates'], results['portfolio_value'], label='CVaR Optimized')
    
    # Add equal-weighted benchmark
    ew_returns = price_data.pct_change().mean(axis=1).dropna()
    ew_values = (1 + ew_returns).cumprod() * results['portfolio_value'][0]
    plt.plot(ew_returns.index, ew_values, label='Equal Weighted')
    
    plt.title('Portfolio Performance Comparison')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value')
    plt.legend()
    plt.grid()
    
    # Drawdown plot
    plt.subplot(2, 2, 2)
    port_returns = pd.Series(results['portfolio_value']).pct_change().dropna()
    cumulative = (1 + port_returns).cumprod()
    peak = cumulative.expanding(min_periods=1).max()
    drawdown = (cumulative - peak) / peak
    drawdown.plot()
    plt.title('Portfolio Drawdown')
    plt.xlabel('Date')
    plt.ylabel('Drawdown')
    plt.grid()
    
    # Weight distribution plot
    plt.subplot(2, 2, 3)
    weights_df = pd.DataFrame(results['weights'], index=results['dates'][1:])
    weights_df.iloc[-1].plot(kind='bar')
    plt.title('Final Portfolio Weights')
    plt.xlabel('Stock')
    plt.ylabel('Weight')
    plt.grid()
    
    # Turnover analysis
    plt.subplot(2, 2, 4)
    results['rebalance_log'].set_index('date')['turnover'].plot()
    plt.title('Rebalancing Turnover')
    plt.xlabel('Date')
    plt.ylabel('Turnover')
    plt.grid()
    
    plt.tight_layout()
    plt.show()
    
    # Print metrics
    metrics = calculate_performance_metrics(
        results['portfolio_value'],
        {'Equal Weighted': ew_values}
    )
    
    print("\nPerformance Metrics:")
    print("="*50)
    for k, v in metrics.items():
        if isinstance(v, float):
            print(f"{k:<30}: {v:.4f}")
        else:
            print(f"{k:<30}: {v}")



In [0]:
# Main execution
if __name__ == "__main__":
    # Configuration
    start_date = '2010-01-01'
    end_date = '2024-12-31'
    
    # Sample 10 liquid S&P 100 stocks for demonstration
    # Replace with full 60 stocks for actual implementation
    sp100_tickers = [
        'AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META',
        'JNJ', 'JPM', 'V', 'PG']
    
    print("Starting Alpha Pods Task A Implementation")
    print("="*50)
    
    try:
        # 1. Fetch and prepare data
        print("\nStep 1: Fetching stock data...")
        price_data = fetch_stock_data(sp100_tickers, start_date, end_date)
        
        # 2. Run backtest
        print("\nStep 2: Running backtest...")
        results = run_backtest(price_data, verbose=True)
        
        # 3. Generate report
        print("\nStep 3: Generating performance report...")
        generate_report(results, price_data)
        
        # 4. Save results to CSV
        print("\nStep 4: Saving results...")
        pd.DataFrame({
            'Date': results['dates'],
            'Portfolio_Value': results['portfolio_value']
        }).to_csv('cvar_index_values.csv', index=False)
        
        print("\nTask A completed successfully!")
        
    except Exception as e:
        print(f"\nError in execution: {str(e)}")
        raise



# Main execution
if __name__ == "__main__":
    # Configuration
    start_date = '2010-01-01'
    end_date = '2024-12-31'
    
    # Sample 60 liquid S&P 100 stocks (replace with actual list)
    sp100_tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'JNJ', 'JPM', 'V', 'PG']
    
    # 1. Fetch data
    price_data = fetch_stock_data(sp100_tickers, start_date, end_date)
    
    # 2. Run backtest
    results = run_backtest(price_data)
    
    # 3. Calculate performance metrics
    metrics = calculate_performance_metrics(results['portfolio_value'], {})
    
    # 4. Create benchmark (equal-weighted)
    ew_returns = price_data.pct_change().mean(axis=1).dropna()
    ew_values = (1 + ew_returns).cumprod()
    
    # 5. Plot results
    plt.figure(figsize=(12, 6))
    plt.plot(results['dates'], results['portfolio_value'], label='CVaR Optimized')
    plt.plot(ew_returns.index, ew_values, label='Equal Weighted')
    plt.title('Portfolio Performance Comparison')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value')
    plt.legend()
    plt.grid()
    plt.show()
    
    # 6. Print metrics
    print("Performance Metrics:")
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}")

In [0]:
# 3. Calculate performance metrics
metrics = calculate_performance_metrics(results['portfolio_value'], {})

# 4. Create benchmark (equal-weighted)
ew_returns = price_data.pct_change().mean(axis=1).dropna()
ew_values = (1 + ew_returns).cumprod()

# 5. Plot results
plt.figure(figsize=(12, 6))
plt.plot(results['dates'], results['portfolio_value'], label='CVaR Optimized')
plt.plot(ew_returns.index, ew_values, label='Equal Weighted')
plt.title('Portfolio Performance Comparison')
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.legend()
plt.grid()
plt.show()

# 6. Print metrics
print("Performance Metrics:")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

In [0]:
# Import required libraries
import numpy as np
import pandas as pd
import yfinance as yf
import cvxpy as cp
from datetime import datetime
import matplotlib.pyplot as plt
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# 1. Data Collection Function (Enhanced)
def fetch_stock_data(tickers, start_date, end_date):
    """
    Fetch historical stock data from Yahoo Finance with error handling
    Args:
        tickers (list): List of stock tickers
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format
    Returns:
        pd.DataFrame: DataFrame with adjusted close prices
    """
    print(f"Downloading data for {len(tickers)} stocks from {start_date} to {end_date}")
    
    try:
        data = yf.download(tickers, start=start_date, end=end_date)['Close']
        
        # Check for failed downloads
        if data.isnull().all().any():
            failed_tickers = data.columns[data.isnull().all()].tolist()
            print(f"Warning: Failed to download data for {failed_tickers} - removing from universe")
            data = data.drop(columns=failed_tickers)
        
        # Clean data
        data = data.ffill().dropna()
        
        if len(data) < 252:  # Less than 1 year of data
            raise ValueError("Insufficient data points after cleaning")
            
        return data
    
    except Exception as e:
        print(f"Error in data download: {str(e)}")
        raise

# 2. CVaR Calculation (Original)
def calculate_cvar(returns, alpha=0.95):
    """
    Calculate Conditional Value-at-Risk (CVaR)
    Args:
        returns (np.array): Array of portfolio returns
        alpha (float): Confidence level (0.95 for 95% CVaR)
    Returns:
        float: CVaR value
    """
    var = np.percentile(returns, 100*(1-alpha))
    cvar = returns[returns <= var].mean()
    return cvar

# 3. Portfolio Optimization (Enhanced)
def optimize_cvar_portfolio(returns, alpha=0.95, max_weight=0.05):
    """
    Optimize portfolio weights to minimize CVaR with fallback
    Args:
        returns (pd.DataFrame): DataFrame of historical returns
        alpha (float): Confidence level for CVaR
        max_weight (float): Maximum weight per stock
    Returns:
        np.array: Optimal weights (falls back to equal weights if optimization fails)
    """
    # Input validation
    returns = returns.fillna(0)
    if len(returns) < 10:
        n = returns.shape[1]
        return np.ones(n) / n
    
    n = returns.shape[1]
    weights = cp.Variable(n)
    
    # Historical scenarios
    scenario_returns = returns.values
    portfolio_returns = scenario_returns @ weights
    
    # CVaR calculation using Rockafellar-Uryasev formulation
    beta = cp.Variable()
    tau = cp.Variable()
    loss = -portfolio_returns
    cvar = tau + (1/(1-alpha)) * cp.mean(cp.pos(loss - tau))
    
    # Problem constraints
    constraints = [
        cp.sum(weights) == 1,
        weights >= 0,
        weights <= max_weight
    ]
    
    # Solve optimization problem with fallback
    problem = cp.Problem(cp.Minimize(cvar), constraints)
    
    try:
        problem.solve(solver=cp.ECOS)
        if weights.value is None:
            raise ValueError("Solver failed to find solution")
            
        # Process weights
        optimized_weights = np.maximum(weights.value, 0)
        optimized_weights = optimized_weights / optimized_weights.sum()
        return optimized_weights
        
    except Exception as e:
        print(f"Optimization warning: {str(e)} - using equal weights")
        return np.ones(n) / n

# 4. Backtesting Engine (Enhanced)
def run_backtest(price_data, rebalance_freq='Q', transaction_cost=0.001):
    """
    Run backtest of CVaR-optimized strategy with robust handling
    Args:
        price_data (pd.DataFrame): DataFrame of historical prices
        rebalance_freq (str): Rebalancing frequency ('Q' for quarterly)
        transaction_cost (float): Transaction cost per trade (10bps = 0.001)
    Returns:
        dict: Backtest results including portfolio values and metrics
    """
    # Calculate and clean returns
    returns = price_data.pct_change().dropna()
    returns = returns.replace([np.inf, -np.inf], np.nan).fillna(0)
    
    dates = returns.index
    rebalance_dates = pd.date_range(dates[0], dates[-1], freq=rebalance_freq)
    
    # Initialize portfolio
    n_stocks = returns.shape[1]
    current_weights = np.ones(n_stocks) / n_stocks
    portfolio_value = [1.0]
    weights_history = []
    
    for i in range(1, len(dates)):
        date = dates[i]
        
        # Rebalance logic
        if date in rebalance_dates:
            # Dynamic lookback window
            lookback_window = min(252, len(returns.loc[:date]))
            lookback_returns = returns.loc[:date].iloc[-lookback_window:]
            
            new_weights = optimize_cvar_portfolio(lookback_returns)
            
            # Apply transaction costs
            turnover = np.sum(np.abs(new_weights - current_weights))
            portfolio_value[-1] *= (1 - transaction_cost * turnover)
            
            current_weights = new_weights
        
        # Daily return calculation
        daily_return = np.dot(returns.iloc[i], current_weights)
        portfolio_value.append(portfolio_value[-1] * (1 + daily_return))
        weights_history.append(current_weights)
    
    return {
        'dates': dates,
        'portfolio_value': portfolio_value,
        'weights': weights_history,
        'returns': returns
    }

# 5. Performance Metrics Calculation (Original)
def calculate_performance_metrics(portfolio_values, benchmark_values, risk_free_rate=0.0):
    """
    Calculate performance metrics
    Args:
        portfolio_values (list): List of portfolio values over time
        benchmark_values (dict): Dictionary of benchmark values
        risk_free_rate (float): Annual risk-free rate
    Returns:
        dict: Dictionary of performance metrics
    """
    # Convert to returns
    port_returns = pd.Series(portfolio_values).pct_change().dropna()
    
    # Annualized return
    annual_return = (1 + port_returns.mean())**252 - 1
    
    # Annualized volatility
    annual_vol = port_returns.std() * np.sqrt(252)
    
    # Sharpe ratio
    sharpe = (annual_return - risk_free_rate) / annual_vol
    
    # Maximum drawdown
    cumulative = (1 + port_returns).cumprod()
    peak = cumulative.expanding(min_periods=1).max()
    drawdown = (cumulative - peak) / peak
    max_drawdown = drawdown.min()
    
    # CVaR
    cvar_95 = calculate_cvar(port_returns)
    
    metrics = {
        'Annual Return': annual_return,
        'Annual Volatility': annual_vol,
        'Sharpe Ratio': sharpe,
        '95% CVaR': cvar_95,
        'Max Drawdown': max_drawdown
    }
    
    # Benchmark comparison if provided
    if benchmark_values:
        for bench_name, bench_values in benchmark_values.items():
            bench_returns = pd.Series(bench_values).pct_change().dropna()
            metrics[f'Excess Return vs {bench_name}'] = metrics['Annual Return'] - ((1 + bench_returns.mean())**252 - 1)
    
    return metrics





# Updated Market Cap Calculation with Proper Error Handling
if __name__ == "__main__":
    # Configuration
    start_date = '2010-01-01'
    end_date = '2024-12-31'
    
    # Sample liquid S&P 100 stocks
    sp100_tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'JNJ', 'JPM', 'V', 'PG', 'HD']
    
    try:
        print("Starting Alpha Pods Task A Implementation")
        print("="*50)
        
        # 1. Fetch price data
        print("\nStep 1: Fetching stock data...")
        price_data = fetch_stock_data(sp100_tickers, start_date, end_date)
        
        # 2. Run backtest
        print("\nStep 2: Running backtest...")
        results = run_backtest(price_data)
        
        # 3. Create benchmarks
        print("\nStep 3: Calculating benchmarks...")
        aligned_returns = price_data.reindex(results['dates']).pct_change().dropna()
        
        # Equal-weighted benchmark
        ew_values = (1 + aligned_returns.mean(axis=1)).cumprod()
        ew_values = ew_values * results['portfolio_value'][0]  # Normalize
        
        # Cap-weighted benchmark - simplified reliable approach
        print("\nCalculating market cap weights...")
        
        # Get current shares outstanding from yfinance
        shares_outstanding = {}
        for ticker in sp100_tickers:
            try:
                stock = yf.Ticker(ticker)
                info = stock.info
                shares = info.get('sharesOutstanding', np.nan)
                if not np.isnan(shares):
                    shares_outstanding[ticker] = shares
                else:
                    print(f"Warning: No shares data for {ticker} - using equal weight")
                    shares_outstanding[ticker] = 1  # Fallback
            except Exception as e:
                print(f"Error getting shares for {ticker}: {str(e)}")
                shares_outstanding[ticker] = 1  # Fallback
        
        # Calculate market cap weights using last available price
        last_prices = price_data.iloc[-1]
        valid_tickers = [t for t in sp100_tickers if t in last_prices and t in shares_outstanding]
        
        if valid_tickers:
            market_caps = {ticker: last_prices[ticker] * shares_outstanding[ticker] 
                          for ticker in valid_tickers}
            total_market_cap = sum(market_caps.values())
            cap_weights = pd.Series({ticker: cap/total_market_cap 
                                   for ticker, cap in market_caps.items()})
            
            # Align weights with our returns data
            cw_returns = (aligned_returns[cap_weights.index] * cap_weights).sum(axis=1)
            cw_values = (1 + cw_returns).cumprod() * results['portfolio_value'][0]
        else:
            print("Warning: No valid tickers for cap weighting - skipping this benchmark")
            cw_values = None
        
        # 4. Calculate metrics
        print("\nStep 4: Calculating performance metrics...")
        benchmarks = {'Equal Weighted': ew_values}
        if cw_values is not None:
            benchmarks['Cap Weighted'] = cw_values
            
        metrics = calculate_performance_metrics(
            results['portfolio_value'],
            benchmarks
        )
        
        # 5. Plot results
        print("\nStep 5: Generating plots...")
        plt.figure(figsize=(14, 7))
        
        # Plot CVaR optimized portfolio
        plt.plot(results['dates'], results['portfolio_value'], 
                label='CVaR Optimized', linewidth=2, color='blue')
        
        # Plot equal-weighted benchmark
        plt.plot(ew_values.index, ew_values, 
                label='Equal Weighted', linestyle='--', color='green')
        
        # Plot cap-weighted benchmark if available
        if cw_values is not None:
            plt.plot(cw_values.index, cw_values,
                    label='Cap Weighted', linestyle=':', color='red')
        
        plt.title('Portfolio Performance Comparison (2010-2024)', fontsize=14)
        plt.xlabel('Date', fontsize=12)
        plt.ylabel('Portfolio Value (Normalized)', fontsize=12)
        plt.legend(fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.7)
        
        # Add performance metrics to plot
        metrics_text = (
            f"CVaR Portfolio:\n"
            f"Ann. Return: {metrics['Annual Return']:.1%}\n"
            f"Ann. Vol: {metrics['Annual Volatility']:.1%}\n"
            f"Sharpe: {metrics['Sharpe Ratio']:.2f}\n"
            f"Max DD: {metrics['Max Drawdown']:.1%}"
        )
        plt.annotate(metrics_text, xy=(0.02, 0.15), xycoords='axes fraction',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        plt.tight_layout()
        plt.show()
        
        # 6. Save results
        print("\nStep 6: Saving results...")
        output_data = {
            'Date': results['dates'],
            'CVaR_Portfolio': results['portfolio_value'],
            'Equal_Weighted': ew_values
        }
        if cw_values is not None:
            output_data['Cap_Weighted'] = cw_values
            
        pd.DataFrame(output_data).to_csv('cvar_index_values.csv', index=False)
        
        # Save weights at last rebalance
        last_rebalance_weights = pd.DataFrame({
            'Ticker': sp100_tickers,
            'Weight': results['weights'][-1]
        })
        last_rebalance_weights.to_csv('final_weights.csv', index=False)
        
        # 7. Print metrics
        print("\nPerformance Metrics:")
        print("="*50)
        for k, v in metrics.items():
            if isinstance(v, float):
                print(f"{k:<25}: {v:.4f}" + ("%" if "Return" in k or "Volatility" in k or "Drawdown" in k else ""))
            else:
                print(f"{k:<25}: {v}")
        
        print("\nTask A completed successfully!")
        
    except Exception as e:
        print(f"\nError in execution: {str(e)}")

In [0]:
# Import required libraries
import numpy as np
import pandas as pd
import yfinance as yf
import cvxpy as cp
from datetime import datetime
import matplotlib.pyplot as plt
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# 1. Data Collection Function (Enhanced)
def fetch_stock_data(tickers, start_date, end_date):
    """
    Fetch historical stock data from Yahoo Finance with error handling
    Args:
        tickers (list): List of stock tickers
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format
    Returns:
        pd.DataFrame: DataFrame with close prices
    """
    print(f"Downloading data for {len(tickers)} stocks from {start_date} to {end_date}")
    
    try:
        data = yf.download(tickers, start=start_date, end=end_date)['Close']
        
        # Check for failed downloads
        if data.isnull().all().any():
            failed_tickers = data.columns[data.isnull().all()].tolist()
            print(f"Warning: Failed to download data for {failed_tickers} - removing from universe")
            data = data.drop(columns=failed_tickers)
        
        # Clean data
        data = data.ffill().dropna()
        
        if len(data) < 252:  # Less than 1 year of data
            raise ValueError("Insufficient data points after cleaning")
            
        return data
    
    except Exception as e:
        print(f"Error in data download: {str(e)}")
        raise


In [0]:
# Updated Market Cap Calculation with Proper Error Handling
if __name__ == "__main__":
    # Configuration
    start_date = '2010-01-01'
    end_date = '2024-12-31'
    
    # Sample liquid S&P 100 stocks
    sp100_tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'JNJ', 'JPM', 'V', 'PG', 'HD']
    
    try:
        print("Starting Alpha Pods Task A Implementation")
        print("="*50)
        
        # 1. Fetch price data
        print("\nStep 1: Fetching stock data...")
        price_data = fetch_stock_data(sp100_tickers, start_date, end_date)


    except Exception as e:
        print(f"\nError in execution: {str(e)}")

In [0]:
price_data

In [0]:


# 2. CVaR Calculation (Original)
def calculate_cvar(returns, alpha=0.95):
    """
    Calculate Conditional Value-at-Risk (CVaR)
    Args:
        returns (np.array): Array of portfolio returns
        alpha (float): Confidence level (0.95 for 95% CVaR)
    Returns:
        float: CVaR value
    """
    var = np.percentile(returns, 100*(1-alpha))
    cvar = returns[returns <= var].mean()
    return cvar

# 3. Portfolio Optimization (Enhanced)
def optimize_cvar_portfolio(returns, alpha=0.95, max_weight=0.05):
    """
    Optimize portfolio weights to minimize CVaR with fallback
    Args:
        returns (pd.DataFrame): DataFrame of historical returns
        alpha (float): Confidence level for CVaR
        max_weight (float): Maximum weight per stock
    Returns:
        np.array: Optimal weights (falls back to equal weights if optimization fails)
    """
    # Input validation
    returns = returns.fillna(0)
    if len(returns) < 10:
        n = returns.shape[1]
        return np.ones(n) / n
    
    n = returns.shape[1]
    weights = cp.Variable(n)
    
    # Historical scenarios
    scenario_returns = returns.values
    portfolio_returns = scenario_returns @ weights
    
    # CVaR calculation using Rockafellar-Uryasev formulation
    beta = cp.Variable()
    tau = cp.Variable()
    loss = -portfolio_returns
    cvar = tau + (1/(1-alpha)) * cp.mean(cp.pos(loss - tau))
    
    # Problem constraints
    constraints = [
        cp.sum(weights) == 1,
        weights >= 0,
        weights <= max_weight
    ]
    
    # Solve optimization problem with fallback
    problem = cp.Problem(cp.Minimize(cvar), constraints)
    
    try:
        problem.solve(solver=cp.ECOS)
        if weights.value is None:
            raise ValueError("Solver failed to find solution")
            
        # Process weights
        optimized_weights = np.maximum(weights.value, 0)
        optimized_weights = optimized_weights / optimized_weights.sum()
        return optimized_weights
        
    except Exception as e:
        print(f"Optimization warning: {str(e)} - using equal weights")
        return np.ones(n) / n

# 4. Backtesting Engine (Enhanced)
def run_backtest(price_data, rebalance_freq='Q', transaction_cost=0.001):
    """
    Run backtest of CVaR-optimized strategy with robust handling
    Args:
        price_data (pd.DataFrame): DataFrame of historical prices
        rebalance_freq (str): Rebalancing frequency ('Q' for quarterly)
        transaction_cost (float): Transaction cost per trade (10bps = 0.001)
    Returns:
        dict: Backtest results including portfolio values and metrics
    """
    # Calculate and clean returns
    returns = price_data.pct_change().dropna()
    returns = returns.replace([np.inf, -np.inf], np.nan).fillna(0)
    
    dates = returns.index
    rebalance_dates = pd.date_range(dates[0], dates[-1], freq=rebalance_freq)
    
    # Initialize portfolio
    n_stocks = returns.shape[1]
    current_weights = np.ones(n_stocks) / n_stocks
    portfolio_value = [1.0]
    weights_history = []
    
    for i in range(1, len(dates)):
        date = dates[i]
        
        # Rebalance logic
        if date in rebalance_dates:
            # Dynamic lookback window
            lookback_window = min(252, len(returns.loc[:date]))
            lookback_returns = returns.loc[:date].iloc[-lookback_window:]
            
            new_weights = optimize_cvar_portfolio(lookback_returns)
            
            # Apply transaction costs
            turnover = np.sum(np.abs(new_weights - current_weights))
            portfolio_value[-1] *= (1 - transaction_cost * turnover)
            
            current_weights = new_weights
        
        # Daily return calculation
        daily_return = np.dot(returns.iloc[i], current_weights)
        portfolio_value.append(portfolio_value[-1] * (1 + daily_return))
        weights_history.append(current_weights)
    
    return {
        'dates': dates,
        'portfolio_value': portfolio_value,
        'weights': weights_history,
        'returns': returns
    }

# 5. Performance Metrics Calculation (Original)
def calculate_performance_metrics(portfolio_values, benchmark_values, risk_free_rate=0.0):
    """
    Calculate performance metrics
    Args:
        portfolio_values (list): List of portfolio values over time
        benchmark_values (dict): Dictionary of benchmark values
        risk_free_rate (float): Annual risk-free rate
    Returns:
        dict: Dictionary of performance metrics
    """
    # Convert to returns
    port_returns = pd.Series(portfolio_values).pct_change().dropna()
    
    # Annualized return
    annual_return = (1 + port_returns.mean())**252 - 1
    
    # Annualized volatility
    annual_vol = port_returns.std() * np.sqrt(252)
    
    # Sharpe ratio
    sharpe = (annual_return - risk_free_rate) / annual_vol
    
    # Maximum drawdown
    cumulative = (1 + port_returns).cumprod()
    peak = cumulative.expanding(min_periods=1).max()
    drawdown = (cumulative - peak) / peak
    max_drawdown = drawdown.min()
    
    # CVaR
    cvar_95 = calculate_cvar(port_returns)
    
    metrics = {
        'Annual Return': annual_return,
        'Annual Volatility': annual_vol,
        'Sharpe Ratio': sharpe,
        '95% CVaR': cvar_95,
        'Max Drawdown': max_drawdown
    }
    
    # Benchmark comparison if provided
    if benchmark_values:
        for bench_name, bench_values in benchmark_values.items():
            bench_returns = pd.Series(bench_values).pct_change().dropna()
            metrics[f'Excess Return vs {bench_name}'] = metrics['Annual Return'] - ((1 + bench_returns.mean())**252 - 1)
    
    return metrics





# Updated Market Cap Calculation with Proper Error Handling
if __name__ == "__main__":
    # Configuration
    start_date = '2010-01-01'
    end_date = '2024-12-31'
    
    # Sample liquid S&P 100 stocks
    sp100_tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'JNJ', 'JPM', 'V', 'PG', 'HD']
    
    try:
        # print("Starting Alpha Pods Task A Implementation")
        # print("="*50)
        
        # # 1. Fetch price data
        # print("\nStep 1: Fetching stock data...")
        # price_data = fetch_stock_data(sp100_tickers, start_date, end_date)
        
        # 2. Run backtest
        print("\nStep 2: Running backtest...")
        results = run_backtest(price_data)
        
        # 3. Create benchmarks
        print("\nStep 3: Calculating benchmarks...")
        aligned_returns = price_data.reindex(results['dates']).pct_change().dropna()
        
        # Equal-weighted benchmark
        ew_values = (1 + aligned_returns.mean(axis=1)).cumprod()
        ew_values = ew_values * results['portfolio_value'][0]  # Normalize
        
        # Cap-weighted benchmark - simplified reliable approach
        print("\nCalculating market cap weights...")
        
        # Get current shares outstanding from yfinance
        shares_outstanding = {}
        for ticker in sp100_tickers:
            try:
                stock = yf.Ticker(ticker)
                info = stock.info
                shares = info.get('sharesOutstanding', np.nan)
                if not np.isnan(shares):
                    shares_outstanding[ticker] = shares
                else:
                    print(f"Warning: No shares data for {ticker} - using equal weight")
                    shares_outstanding[ticker] = 1  # Fallback
            except Exception as e:
                print(f"Error getting shares for {ticker}: {str(e)}")
                shares_outstanding[ticker] = 1  # Fallback
        
        # Calculate market cap weights using last available price
        last_prices = price_data.iloc[-1]
        valid_tickers = [t for t in sp100_tickers if t in last_prices and t in shares_outstanding]
        
        if valid_tickers:
            market_caps = {ticker: last_prices[ticker] * shares_outstanding[ticker] 
                          for ticker in valid_tickers}
            total_market_cap = sum(market_caps.values())
            cap_weights = pd.Series({ticker: cap/total_market_cap 
                                   for ticker, cap in market_caps.items()})
            
            # Align weights with our returns data
            cw_returns = (aligned_returns[cap_weights.index] * cap_weights).sum(axis=1)
            cw_values = (1 + cw_returns).cumprod() * results['portfolio_value'][0]
        else:
            print("Warning: No valid tickers for cap weighting - skipping this benchmark")
            cw_values = None
        
        # 4. Calculate metrics
        print("\nStep 4: Calculating performance metrics...")
        benchmarks = {'Equal Weighted': ew_values}
        if cw_values is not None:
            benchmarks['Cap Weighted'] = cw_values
            
        metrics = calculate_performance_metrics(
            results['portfolio_value'],
            benchmarks
        )
        
        # 5. Plot results
        print("\nStep 5: Generating plots...")
        plt.figure(figsize=(14, 7))
        
        # Plot CVaR optimized portfolio
        plt.plot(results['dates'], results['portfolio_value'], 
                label='CVaR Optimized', linewidth=2, color='blue')
        
        # Plot equal-weighted benchmark
        plt.plot(ew_values.index, ew_values, 
                label='Equal Weighted', linestyle='--', color='green')
        
        # Plot cap-weighted benchmark if available
        if cw_values is not None:
            plt.plot(cw_values.index, cw_values,
                    label='Cap Weighted', linestyle=':', color='red')
        
        plt.title('Portfolio Performance Comparison (2010-2024)', fontsize=14)
        plt.xlabel('Date', fontsize=12)
        plt.ylabel('Portfolio Value (Normalized)', fontsize=12)
        plt.legend(fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.7)
        
        # Add performance metrics to plot
        metrics_text = (
            f"CVaR Portfolio:\n"
            f"Ann. Return: {metrics['Annual Return']:.1%}\n"
            f"Ann. Vol: {metrics['Annual Volatility']:.1%}\n"
            f"Sharpe: {metrics['Sharpe Ratio']:.2f}\n"
            f"Max DD: {metrics['Max Drawdown']:.1%}"
        )
        plt.annotate(metrics_text, xy=(0.02, 0.15), xycoords='axes fraction',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        plt.tight_layout()
        plt.show()
        
        # 6. Save results
        print("\nStep 6: Saving results...")
        output_data = {
            'Date': results['dates'],
            'CVaR_Portfolio': results['portfolio_value'],
            'Equal_Weighted': ew_values
        }
        if cw_values is not None:
            output_data['Cap_Weighted'] = cw_values
            
        pd.DataFrame(output_data).to_csv('cvar_index_values.csv', index=False)
        
        # Save weights at last rebalance
        last_rebalance_weights = pd.DataFrame({
            'Ticker': sp100_tickers,
            'Weight': results['weights'][-1]
        })
        last_rebalance_weights.to_csv('final_weights.csv', index=False)
        
        # 7. Print metrics
        print("\nPerformance Metrics:")
        print("="*50)
        for k, v in metrics.items():
            if isinstance(v, float):
                print(f"{k:<25}: {v:.4f}" + ("%" if "Return" in k or "Volatility" in k or "Drawdown" in k else ""))
            else:
                print(f"{k:<25}: {v}")
        
        print("\nTask A completed successfully!")
        
    except Exception as e:
        print(f"\nError in execution: {str(e)}")